In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"

In [9]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
import os
from langchain.text_splitter import CharacterTextSplitter

In [10]:
import os
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] =

In [11]:
directory_path = '/content/drive/My Drive/HowToPlayWeiss'
chunk_size = 1000
chunk_overlap = 200
all_chunks = []

for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):  # Process only .txt files
        file_path = os.path.join(directory_path, filename)

        # Load the text file
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()
        start = 0
        while start < len(data):
            end = min(start + chunk_size, len(data))
            chunk = data[start:end]
            all_chunks.append(chunk)
            start += (chunk_size - chunk_overlap)

print(f"Total chunks: {len(all_chunks)}")


Total chunks: 1044


In [12]:
from langchain.schema import Document
embeddings = OpenAIEmbeddings()
all_documents = [Document(page_content=chunk) for chunk in all_chunks]
vectorstore = FAISS.from_documents(all_documents, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [19]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [22]:
query = """
Consider the effect: 【AUTO】 [Put 1 card from your hand into your waiting room] When this card is placed on the stage from your hand, you may pay the cost. If you do, reveal the top card of your deck, choose 1 level X or lower character in your waiting room, and return it to your hand. X is equal to the level of the revealed card. (Climax are regarded as level 0. Return the revealed card to its original place).
When might a player want to or want not to activate this effect? What zones of their board should they check before deciding whether not to activate it? """
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"A player might choose to activate this effect when they want to retrieve a specific character card from their waiting room to their hand, especially if that card has a powerful effect or can contribute to their game strategy. The level of the card they can retrieve depends on the level of the card they reveal from the top of their deck, so they might also consider the potential levels of cards in their deck before deciding to activate this effect.\n\nBefore making a decision on whether to activate this effect, a player should review the following areas of their game board:\n\n1. Their hand: They need to have at least one card they are willing to put into the waiting room to pay the cost of this effect.\n\n2. Their deck: They should consider the potential levels of cards in their deck, as this will determine the level of the card they can retrieve from the waiting room.\n\n3. Their waiting room: They need to have a character card of the appropriate level that they want to retrieve.\n\n

In [24]:
query = """You are an expert at weiss schawrz.
You are playing the set Hololive.
It is your main phase . You have placed card \'Pavolia Reine, Time Just for the Two... \ on stage.
You are choosing whether it is in your favor to activate the effect:
【AUTO】 [Put 1 card from your hand into your waiting room] When this card is placed on the stage from your hand, you may pay the cost. If you do, reveal the top card of your deck, choose 1 level X or lower character in your waiting room, and return it to your hand. X is equal to the level of the revealed card. (Climax are regarded as level 0. Return the revealed card to its original place)

Your deck runs 10 level 3 cards, 8 cx cards, 12 level 1 cards, and 20 level 0 characters(climax cards are NOT characters).
You stage is
Center left: none
Center Middle: Pavolia Reine, Time Just for the Two...
Center right: none
Back left: none
Back right: none
You hand is : 2 x Miko Sakura, Barely Awake Eye(Level 3), 1 x Calliope Mori, A Step Towards the Future(Level 1), nothing else
Your waiting room is: 1 x Miko Sakura, Barely Awake Eye(Level 3)
Your deck has 32 cards in it
Your stock has 0 cards in it
Your memory has 0 cards in it
The first character of the answer should be a definitive 1 for Yes or 0 for No followed by explanation if neede
"""
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"1\n\nYes, you should activate the effect of 'Pavolia Reine, Time Just for the Two...'. The reason is that you have a good chance of revealing a card that allows you to retrieve a character from your waiting room. Given the composition of your deck, you have a 62.5% chance of revealing a Level 1 or Level 0 card, which would allow you to retrieve the 'Miko Sakura, Barely Awake Eye' from your waiting room. Even if you reveal a Level 3 card or a Climax card (which is considered Level 0), you can still retrieve a Level 0 character from your waiting room. Since your waiting room only contains a Level 3 character, you would not be able to retrieve it if a Climax card is revealed, but the odds are in your favor. Furthermore, activating the effect would not cost you any stock, only a card from your hand, which you have."